In [19]:
# Add parent directory to path to import py222
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [12]:
def load_dataset_split(data_dir: Path, split: str):
    # Load a dataset split (train/test/val)
    split_dir = data_dir / split
    if not split_dir.exists():
        raise FileNotFoundError(f"Split {split} not found in {data_dir}")

    # Load metadata
    with open(split_dir / "dataset.json", "r") as f:
        metadata = json.load(f)

    # Load arrays
    inputs = np.load(split_dir / "all__inputs.npy")
    labels = np.load(split_dir / "all__labels.npy")
    group_indices = np.load(split_dir / "all__group_indices.npy")
    puzzle_indices = np.load(split_dir / "all__puzzle_indices.npy")
    puzzle_identifiers = np.load(split_dir / "all__puzzle_identifiers.npy")

    return {
        'metadata': metadata,
        'inputs': inputs,
        'labels': labels,
        'group_indices': group_indices,
        'puzzle_indices': puzzle_indices,
        'puzzle_identifiers': puzzle_identifiers,
    }

In [15]:
data = load_dataset_split(Path("../data/cube-2-by-2"), "test")

In [16]:
df = pd.DataFrame({
    'inputs': [data['inputs'][i].tolist() for i in range(len(data['inputs']))],
    #'labels': [data['labels'][i].tolist() for i in range(len(data['labels']))],
    'labels': [[x for x in data['labels'][i] if x != 0] for i in range(len(data['labels']))],
    'puzzle_indices': data['puzzle_indices'][:-1],
})

In [17]:
colors = ['Y','R','G','O','B','W']
translate = [0, 3, 2, 5, 1, 4]
# move indices
moveInds = { \
  "U": 0, "U'": 1, "U2": 2, "R": 3, "R'": 4, "R2": 5, "F": 6, "F'": 7, "F2": 8, \
  "D": 9, "D'": 10, "D2": 11, "L": 12, "L'": 13, "L2": 14, "B": 15, "B'": 16, "B2": 17, \
  "x": 18, "x'": 19, "x2": 20, "y": 21, "y'": 22, "y2": 23, "z": 24, "z'": 25, "z2": 26 \
}
inverted = {(v + 1): k for k, v in moveInds.items()}
inverted[0] = '0'
print(inverted)
sample = data['inputs'][0]
label = data['labels'][0]
print(sample-1)
print(''.join([colors[i-1] for i in sample]))
state = ''.join([colors[translate[i-1]] for i in sample])
sol = ''.join([str(inverted[i]) + " " for i in label])
print(state)
print(sol)

{1: 'U', 2: "U'", 3: 'U2', 4: 'R', 5: "R'", 6: 'R2', 7: 'F', 8: "F'", 9: 'F2', 10: 'D', 11: "D'", 12: 'D2', 13: 'L', 14: "L'", 15: 'L2', 16: 'B', 17: "B'", 18: 'B2', 19: 'x', 20: "x'", 21: 'x2', 22: 'y', 23: "y'", 24: 'y2', 25: 'z', 26: "z'", 27: 'z2', 0: '0'}
[2 2 5 5 0 3 0 2 3 4 1 4 0 2 3 4 0 1 4 5 1 1 3 5]
GGWWYOYGOBRBYGOBYRBWRROW
GGBBYWYGWRORYGWRYORBOOWB
F2 U2 R' F U F2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


In [20]:
import py222

sample_idx = 0
cube_input = data['inputs'][sample_idx]
labels = data['labels'][sample_idx]

py222_state = cube_input - 1
print(py222_state)
py222.printCube(py222_state)

move_names = {0: "U", 1: "U'", 2: "U2", 3: "R", 4: "R'", 5: "R2", 6: "F", 7: "F'", 8: "F2"}

moves = [int(m) - 1 for m in labels if m != 0]
sol_str = ' '.join([move_names[m] for m in moves])
print(sol_str)

test_state = py222_state
for m in moves:
    test_state = py222.doMove(test_state, m)
print(test_state)

[2 2 5 5 0 3 0 2 3 4 1 4 0 2 3 4 0 1 4 5 1 1 3 5]
      ┌──┬──┐
      │ 2│ 2│
      ├──┼──┤
      │ 5│ 5│
┌──┬──┼──┼──┼──┬──┬──┬──┐
│ 0│ 1│ 3│ 4│ 0│ 3│ 1│ 1│
├──┼──┼──┼──┼──┼──┼──┼──┤
│ 4│ 5│ 1│ 4│ 0│ 2│ 3│ 5│
└──┴──┼──┼──┼──┴──┴──┴──┘
      │ 0│ 2│
      ├──┼──┤
      │ 3│ 4│
      └──┴──┘
F2 U2 R' F U F2
[0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 5 5 5 5]


In [27]:
import magiccube

solved_py222 = py222.initState()
solved_magic = magiccube.Cube(2)
test_py222 = py222.doMove(solved_py222.copy(), 3)
test_magic = magiccube.Cube(2)
test_magic.rotate("R")
color_map = {0: 'W', 1: 'R', 2: 'G', 3: 'Y', 4: 'O', 5: 'B'}

def py222_to_magiccube(state):
    color_map = {0: 'W', 1: 'R', 2: 'G', 3: 'Y', 4: 'O', 5: 'B'}
    
    # Reorder stickers from px222 to magiccube notation
    magic_order = (
        list(range(0, 4)) +
        list(range(16, 20)) +
        list(range(8, 12)) +
        list(range(4, 8)) +
        list(range(20, 24)) +
        list(range(12, 16))
    )
    return ''.join([color_map[state[i]] for i in magic_order])

converted = py222_to_magiccube(solved_py222)
test_cube = magiccube.Cube(2, converted)

converted_scrambled = py222_to_magiccube(py222_state)
cube = magiccube.Cube(2, converted_scrambled)
print(cube)
cube.rotate("F2 U2 R' F U F2")
print(cube)

       G  G             
       B  B             
 W  R  Y  O  W  Y  R  R 
 O  B  R  O  W  G  Y  B 
       W  G             
       Y  O             

       W  W             
       W  W             
 O  O  G  G  R  R  B  B 
 O  O  G  G  R  R  B  B 
       Y  Y             
       Y  Y             



In [28]:
str(cube).replace(" ", "").replace("\n", "")

'WWWWOOGGRRBBOOGGRRBBYYYY'

In [29]:
df.head()

,inputs,labels,puzzle_indices
0,"[3, 3, 6, 6, 1, 4, 1, 3, 4, 5, 2, 5, 1, 3, 4, ...","[9, 3, 5, 7, 1, 9]",0
1,"[2, 3, 1, 5, 3, 2, 6, 4, 3, 4, 3, 5, 2, 1, 4, ...","[1, 5, 8, 5, 2, 4]",1
2,"[5, 4, 2, 1, 6, 2, 5, 3, 6, 2, 2, 1, 3, 6, 4, ...","[1, 6, 3, 7, 2, 5, 8, 1, 7]",2
3,"[3, 3, 6, 4, 5, 4, 5, 1, 4, 3, 1, 1, 6, 6, 4, ...","[1, 4, 2, 7, 2, 9, 6, 8, 6]",3
4,"[1, 2, 2, 6, 1, 3, 4, 5, 3, 5, 2, 3, 6, 5, 4, ...","[8, 1, 4, 7, 5, 7]",4


In [30]:
df_inputs = pd.DataFrame(df['inputs'].tolist())
df_outputs = pd.DataFrame(df['labels'].tolist())

In [31]:
df_inputs.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,3,3,6,6,1,4,1,3,4,5,...,4,5,1,2,5,6,2,2,4,6
1,2,3,1,5,3,2,6,4,3,4,...,4,6,1,5,5,4,1,6,2,6
2,5,4,2,1,6,2,5,3,6,2,...,4,1,3,4,5,1,3,4,5,6
3,3,3,6,4,5,4,5,1,4,3,...,4,3,5,2,5,2,2,1,2,6
4,1,2,2,6,1,3,4,5,3,5,...,4,3,6,1,5,4,4,2,1,6


In [32]:
df_inputs.describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.0,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.0
mean,3.267000,3.292000,3.222000,3.232000,3.26000,3.284000,3.248000,3.285000,3.328000,3.324000,...,4.0,3.465000,3.359000,3.287000,5.0,3.259000,3.360000,3.211000,3.207000,6.0
std,1.742627,1.701656,1.712205,1.706191,1.71445,1.710634,1.687407,1.690157,1.682036,1.707612,...,0.0,1.712807,1.688482,1.702801,0.0,1.674133,1.648975,1.695687,1.678383,0.0
min,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,4.0,1.000000,1.000000,1.000000,5.0,1.000000,1.000000,1.000000,1.000000,6.0
25%,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,...,4.0,2.000000,2.000000,2.000000,5.0,2.000000,2.000000,2.000000,2.000000,6.0
50%,3.000000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,...,4.0,3.000000,3.000000,3.000000,5.0,3.000000,3.000000,3.000000,3.000000,6.0
75%,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,...,4.0,5.000000,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,6.0
max,6.000000,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000,6.000000,6.000000,6.000000,...,4.0,6.000000,6.000000,6.000000,5.0,6.000000,6.000000,6.000000,6.000000,6.0


In [33]:
df_outputs.head()

,0,1,2,3,4,5,6,7,8,9
0,9,3,5.0,7.0,1.0,9.0,NaN,NaN,NaN,NaN
1,1,5,8.0,5.0,2.0,4.0,NaN,NaN,NaN,NaN
2,1,6,3.0,7.0,2.0,5.0,8.0,1.0,7.0,NaN
3,1,4,2.0,7.0,2.0,9.0,6.0,8.0,6.0,NaN
4,8,1,4.0,7.0,5.0,7.0,NaN,NaN,NaN,NaN


In [34]:
df_outputs.describe()

,0,1,2,3,4,5,6,7,8,9
count,1000.000000,1000.000000,999.000000,997.000000,980.000000,904.000000,797.000000,679.000000,442.000000,114.000000
mean,3.893000,5.113000,4.669670,5.167503,4.845918,5.108407,4.897114,5.142857,5.248869,4.824561
std,2.517119,2.482417,2.574538,2.589178,2.586153,2.623553,2.553833,2.659968,2.551343,2.816656
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000
50%,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.500000
75%,6.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,8.000000,7.750000,8.000000
max,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [38]:
count = df_outputs.iloc[:, 9].notna().sum()
print(count)

114
